# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import nltk, pandas as pd, rake_nltk, plotly.express as px

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

## **Keyword Extraction Algorithms (KEAs)**

<span style="color:black">In this notebook, you will practice measuring the performance of keyword extraction algorithms (KEAs). Comparing two algorithms can be automated, but there are still many ways to compare them. However, if you are evaluating the performance of a single KEA, experts are needed to evaluate its output in the absence of labels.
    
<div style="margin-top: 20px; margin-bottom: 20px;">
<details style="border: 2px solid #ddd; margin-bottom: -2px;">
    <summary style="padding: 12px 15px; cursor: pointer; background-color: #eee;">
        <div id="button" style="padding: 0px;">
            <font color=#B31B1B>▶ </font> 
            <b> Why Measure Performance of KEAs? </b>
        </div>
    </summary>
    <div id="button_info" style="padding:10px"> 
        There are various reasons for measuring performance of KEAs. Below are several common reasons:
            
1. You might want to transition from a slower or poorer KEA to a faster or improved KEA in a production environment. 
1. You might want to try a different set of hyperparameters on an existing KEA. 
1. You might want to combine several KEAs and evaluate the diversity of their outputs. 
1. You might want to have several KEAs vote on the keywords they extract. 
    </div>
</details>
</div>
<span style="color:black">Begin by loading the Gutenberg stopwords corpora and then the raw text from "Alice's Adventures in Wonderland," from which you will extract keywords.

<span style="color:black"><b>Note:</b> The PKE package requires that NLTK stopwords are downloaded first.
 
 

In [ ]:
_ = nltk.download(['gutenberg', 'stopwords'], quiet=True)
import pke       # import pke after NLTK's stopwords are loaded
sDoc = nltk.corpus.gutenberg.raw(fileids='carroll-alice.txt')
print(sDoc[:300])

## **Extract Keywords From KEAs**

<span style="color:black">Before you can empirically measure the performance of the keyword extraction algorithm (KEA), you must extract some keywords. Typically, you want to compare two KEAs based on an equal number of top extracted words. The comparison would be less sensible, for example, if KEA A extracts 100 words, but KEA B extracts an arbitrary number of keywords. More extracted keywords increases the coverage of KEA A.

<span style="color:black">Extract the top relevant keywords using PKE's TextRank algorithm. You can experiment with different hyperparameters if you wish to try to improve the extraction.

In [ ]:
def GetTextRank(sDoc:'document'='', n:'top n keywords'=100, w:'window'=5) -> [(str, float)]:
    '''Retrieves top n keywords with scores in order decreasing importance'''
    KEA = pke.unsupervised.TextRank()                             # instantiate a TextRank object
    KEA.load_document(input=sDoc, language='en')                  # 1. load and pre-process a document
    KEA.candidate_selection(pos={'NOUN', 'PROPN', 'ADJ'})         # 2. create a graph of candidate phrases (as graph nodes)
    KEA.candidate_weighting(window=w, pos=None, top_percent=None, normalized=True)  # 3. compute importance ranks of phrases
    return KEA.get_n_best(n=n, redundancy_removal=True, stemming=False)

LTsnKwd = GetTextRank(sDoc)
LTsnKwd[:3]

In [ ]:
n = 100
dfTR = pd.DataFrame(LTsnKwd, columns=['Keyword', 'Score'])
dfTR.set_index('Keyword').round(3).T

<span style="color:black"> Now extract the same number of top keywords with RAKE, another KEA.

In [ ]:
def GetRAKE(sDoc:'document'='', n:'top n keywords'=100, l:'max_length'=3) -> set():
    '''Retrieves top n keywords with scores in order decreasing importance'''
    r = rake_nltk.Rake(language="english", max_length=l, min_length=1)  # instantiate Rake object
    r.extract_keywords_from_text(sDoc)                                  # build a list of candidate phrases
    return r.get_ranked_phrases_with_scores()[:n]

LTnsKwd = GetRAKE(sDoc)
LTnsKwd[:3]

In [ ]:
dfRK = pd.DataFrame(LTnsKwd, columns=['score', 'Keyword'])
dfRK.set_index('Keyword').round(3).T

<span style="color:black"> If you compare the outputs of the two KEAs, you can quickly notice phrases from both algorithms containing the word `'little'` and a few more matching words. However, it is difficult to say how many phrases match exactly because even matched phrases are not aligned due to varying relative ranking. 

## **Compare Using Jaccard Similarity**

<span style="color:black">A popular metric for comparing sets of *any* objects is [Jaccard similarity](https://scikit-learn.org/stable/modules/model_evaluation.html#jaccard-similarity-score). This metric is relatively simple to implement, and you will apply this to the keywords from the two KEAs. 
<div style="margin-top: 20px; margin-bottom: 20px;">
<details style="border: 2px solid #ddd; margin-bottom: -2px;">
    <summary style="padding: 12px 15px; cursor: pointer; background-color: #eee;">
        <div id="button" style="padding: 0px;">
            <font color=#B31B1B>▶ </font> 
            <b> More About: </b>Jaccard Similarity
        </div>
    </summary>
    <div id="button_info" style="padding:10px"> 
       Jaccard Similarity is among the most primitive metrics as it simply counts the fraction of matching elements with little regard for their structure or data type. As long as elements can be compared (on the basis of *equal* or *not equal*) and counted, you can apply Jaccard similarity. As the name "similarity" suggests, a higher value implies greater similarity between the two sets. The formula for sets $A,B$:
 
$$J(A,B):=\frac{|A\cap B|}{|A \cup B|}$$
 
where $|A|$ is the *cardinality* of the set $A$, i.e. the number of elements in $A$.
    </div>
</details>
</div>

In [ ]:
LsTR, LsRK = dfTR.Keyword.tolist(), dfRK.Keyword.tolist()

SsTR, SsRK = set(LsTR), set(LsRK)
nFrac = len(SsTR & SsRK) / len(SsTR | SsRK)             # alternative: A.intersection(B), A.union(B)
print(f'{len(SsTR & SsRK)}/{len(SsTR | SsRK)}={nFrac:.3f}:', '|'.join(SsTR & SsRK))

<span style="color:black">There is about 3% similarity for the two sets of 100 words, i.e., 3% @100. As the number of extracted keywords changes, so does the fraction of matches, which can improve or degrade. By itself, this value is not very informative. It is more meaningful when you start altering the hyperparameters or trying different KEAs. Then this performance metric can be used to automate KEA selection.

<span style="color:black">You can observe that many words repeat in the keywords extracted by both KEAs, so it may make sense to compare the match of words in keywords. Additionally, decomposing phrases into words helps in comparing phrases such as `'I fly home'` to `'I 🛩️ 🏠'`, which are not equal but are lexically and semantically similar. Jaccard similarity, based on words in the phrases, implies 1/5 similarity, since only one word (`I`) is in the intersection of the two sets of words and five words are in their union (`I`, `fly`, `home`, 🛩️, 🏠). 
 
<span style="color:black"> Join each set of keywords, split the resulting string into individual word tokens, and repeat the Jaccard similarity calculation for the two sets of words where duplicated words are removed.

In [ ]:
SsTR, SsRK = set(' '.join(LsTR).split()), set(' '.join(LsRK).split())  # convert keyword lists to sets

# Jaccard similarity (fraction of overlap): size of intersection(A,B) divide by size of the union(A,B)
nFrac = len(SsTR & SsRK)/len(SsTR | SsRK)
sMatches = '|'.join(SsTR & SsRK)

print(f'{len(SsTR & SsRK)}/{len(SsTR | SsRK)}={nFrac:.3f}:', sMatches)

<span style="color:black">The resulting similarity is about a 10% similarity for the top 100 retrieved words.

## **`StemPhrase()`**

<span style="color:black"> Sometimes the differences among keywords is due to word morphology, and so, it may be useful to investigate the morphology of words in the extracted keywords. Phrases such as `'little key'` and `'little keys'`, which only differ by the character `'s'`, might be considered the same. Thus, the keyword similarity metric can be improved if the words are preprocessed with a stemmer or lemmatizer. 

The UDF `StemPhrase()` below takes a keyword, splits it into words, and re-joins the stemmed words into a phrase again. The `StemPhrases()` repeats the same task for each string keyword in its list argument.
    
The example below shows how a phase `'little golden keys'` is standardized to `'littl golden key'`. Thus any existing plural and singular forms of this phrase are collapsed into one, which can improve representativeness of the underlying text. Also, note that `'miserable mock turtle'` is replaced with `'miser mock turtl'`, where the word `'miser'` has a different meaning from the original `'miserable'`. This may not be a problem, if the phrase still helps to represent the underlying text. However, it may be problematic if we are comparing two documents: one containing words `'miserable'` and one containing words `'miser'`. In such cases, the texts are both represented by the stemmed word `'miser'` and do become more similar. One should always evaluate the resulting keywords for their representativeness to the underlying text. We could add `'miserable'` to some stop lists, but manually maintaining large stop lists can be time consuming and costly. You should try to derive more generalizable rules and algorithms, which are immune to occasional errors.

In [ ]:
StemPhrase  = lambda   sPhrase: ' '.join(nltk.PorterStemmer().stem(s) for s in sPhrase.split())
StemPhrases = lambda LsPhrases: [StemPhrase(s) for s in LsPhrases]  # stem each word in each phrase
StemPhrases(['little golden keys', 'miserable mock turtle'])

<span style="color:black">Now apply `StemPhrases()` to standardize the phrases before calculating Jaccard similarity. 

In [ ]:
SsTR, SsRK = set(StemPhrases(LsTR)), set(StemPhrases(LsRK))
nFrac = len(SsTR & SsRK)/len(SsTR | SsRK)    # = intersection size/union size, or |A ∩ B| / |A ∪ B|
print(f'{len(SsTR & SsRK)}/{len(SsTR | SsRK)}={nFrac:.3f}', SsTR & SsRK)

<span style="color:black">No improvement in the metric is noticed.

<span style="color:black">In certain situations, you might consider the phrases `'poor little alice'` and `'little poor alice'` to be the same, even though they are not equivalent as strings. If you compare these as sets of words instead, both turn out to be `{'poor', 'little', 'alice'}`. 
 
<span style="color:black">Try this approach to display keywords (sets of words) which match among the outputs of TextRank KEA and RAKE KEA. 

In [ ]:
LSsTR, LSsRK = [set(s.split()) for s in SsTR], [set(s.split()) for s in SsRK]  # create a set of words from each keyword
[k for k in LSsTR if k in LSsRK]                           # compare matching keywords (as sets of words)

<span style="color:black">While this still only produces 6 matching keyphrases (out of 180), you just learned yet another approach in comparing outputs of two KEAs, which can be used to compare documents in a corpus, perform document searches, compress documents and in a wide variety of other NLP tasks.

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**
Now, equipped with these concepts and tools, you will tackle a few related tasks.

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

## Task 1

Write a function `JacSim(A,B)`, which takes any two container objects, converts them to sets and computes their Jaccard similarity. Then try this on sets (or lists) of words from `'I fly home'` and `'I 🛩️ 🏠'`, which should return 0.2. Also, handle the zero denominator case by returning a zero similarity. 

<b>Hint:</b> See the Jaccard similarity computation above. Wrap it into a function.

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
def JacSim(A, B) -> float:
    A, B = set(A), set(B)  # ensure that iterables are sets
    return 0 if len(A | B)==0 else len(A & B)/len(A | B)  # Jaccard similarity for sets A and B

A, B = 'I fly home'.split(), 'I 🛩️ 🏠'.split()
JacSim(A,B)
</pre>
</details> 
</font>
<hr>

## Task 2
 
Use `GetTextRank` and `GetRAKE` with default parameters to extract the lists of top 100 words from `sDoc` in order of decreasing scores. Drop the scores and save the resulting lists of strings as `LsTR` and `LsRK`, correspondingly.

<b>Hint:</b> You can use list comprehension to iterate over score/keyword pairs, but return the keyword only. Note that one function returns score/keyword and the other returns keyword/score.

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre>
LsTR = [s for s, n in GetTextRank(sDoc)] # create a set of top 100 keywords with TextRank
LsRK = [s for n, s in GetRAKE(sDoc)]     # create a set of top 100 keywords with RAKE
LsTR[:3]  # shows a sample from extracted keywords
LsRK[:3]  # shows a sample from extracted keywords
</pre>
</details> 
</font>
<hr>

## Task 3

For `n` varying from 0 to the length of `LsTR`, retrieve top `n` words of `LsTR` and of `LsRK` and compute their Jaccard similarity. As a bonus, try plotting Jaccard similarity for each `n` with [`plotly.express.scatter()`](https://plotly.com/python/line-and-scatter/) plot. 

Notice that two lists have no common keywords for about 60 top keywords and reach about 3% similarity with top 100 keywords. This implies that RAKE and TextRank would be poor substitutions for each other, but could work together as an ensemble to extract a more diverse set of keywords.

<b>Hint:</b> You can use list comprehension iterate over increasing values of <code>n</code>.

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre>
df1 = pd.DataFrame([JacSim(LsTR[:n], LsRK[:n]) for n in range(len(LsTR))], columns=['JacSim'])
df1.index.name = 'n, the number of top keywords'

cols = (df1.JacSim*200).astype(int).values
fig = px.scatter(df1, title='Jaccard Similarity for TextRanke@n and RAKE@n', height=300, color=cols);
fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))
</pre>
</details> 
</font>
<hr>